In [1]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install pycocotools

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-qhztfm3c
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-qhztfm3c
  Resolved https://github.com/facebookresearch/detectron2.git to commit c69939aa85460e8135f40bce908a6cddaa73065f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.0 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

In [2]:
# Mounting drive so that shared folder can be used
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os
import cv2
import random
import json

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [4]:
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer

root_path = '/content/drive/MyDrive/Retail Product Checkout v35i'
#Reading train and test data in COCO format
register_coco_instances("rpc_train", {}, os.path.join(root_path,"train/_annotations.coco.json"), os.path.join(root_path,"train"))
register_coco_instances("rpc_test", {}, os.path.join(root_path,"test/_annotations.coco.json"), os.path.join(root_path,"test"))

rpc_train_metadata = MetadataCatalog.get("rpc_train")
rpc_test_metadata = MetadataCatalog.get("rpc_test")


In [5]:
# Create Default Predictor
from detectron2.engine import DefaultTrainer,DefaultPredictor
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("rpc_train",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
cfg.MODEL.DEVICE = "cuda" # use cpu if gpu isnt available

cfg.MODEL.WEIGHTS = os.path.join('/content/drive/MyDrive/output','model_final.pth')

# cfg.MODEL.WEIGHTS = os.path.join(root_path,"smart_billing_model/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set the testing threshold for this model. For a model trained for 1000 epochs set to 0.3 to see results.
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.3    # NMS threshold
cfg.DATASETS.TEST = ("rpc_test", )
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
predictor = DefaultPredictor(cfg)


[12/19 07:03:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be e

[12/19 07:04:07 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/output/model_final.pth ...


In [6]:
import matplotlib.pyplot as plt
from collections import defaultdict
from tabulate import tabulate
import random
import cv2
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer,DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog

py_predictor = DefaultPredictor(cfg)

# Visualizing detections in images randomly sampled from the test set
dataset_dicts = DatasetCatalog.get("rpc_test")

# List of product names from the metadata
product_names = list(rpc_test_metadata.thing_classes)

# Function to create a list of detected instances based on predicted classes
def create_list(instances):
    det_items = [product_names[i] for i in instances]
    return det_items

# Function to print a bill of detected products with their quantities
def print_bill(instances):
    bill = defaultdict(int)
    for i in instances:
        bill[i] += 1
    l = []
    for key, value in bill.items():
        l.append([product_names[int(key)], value])
    # Create and print table using tabulate
    table = tabulate(l, headers=['Product Name', 'Quantity'], tablefmt='orgtbl')
    print(table)


# Iterate through random samples from the test set
for d in random.sample(dataset_dicts, 5):
    # Read the image
    im = cv2.imread(d["file_name"])

    print('\n')
    print(d["file_name"])

    # Get predictions from the model
    outputs = py_predictor(im)

    # Visualize the instance predictions
    v = Visualizer(im[:, :, ::-1], metadata=rpc_test_metadata, scale=1)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Display the image using matplotlib
    plt.figure(figsize=(30, 30))
    plt.imshow(v.get_image()[:, :, ::-1])  # Convert BGR to RGB
    plt.axis('off')  # Hide the axis
    plt.show()

    # Get the predicted class labels
    items = outputs["instances"].pred_classes.cpu().detach().numpy()
    items = list(items)

    # Print the predicted items and generate a bill
    print(items)
    print_bill(items)


In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Set up the evaluator and test loader
evaluator = COCOEvaluator("rpc_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "rpc_test")

# Run inference and evaluation
metrics = inference_on_dataset(py_predictor.model, val_loader, evaluator)
print(metrics)  # This will print mAP and other metrics


WARNING [12/19 07:04:23 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [12/19 07:04:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/19 07:04:23 d2.data.datasets.coco]: Loaded 51 images in COCO format from /content/drive/MyDrive/Retail Product Checkout v35i/test/_annotations.coco.json
[12/19 07:04:23 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|      Er       | 0            | 4D_medical_.. | 25           | Let-green_a.. | 15           |
|     X-men     | 17           |   aquafina    | 16           |     cart      | 24           |
|   life-buoy   | 17           |   luong_kho   | 18           |     milo

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[12/19 07:04:27 d2.evaluation.evaluator]: Inference done 11/51. Dataloading: 0.1035 s/iter. Inference: 0.1277 s/iter. Eval: 0.0003 s/iter. Total: 0.2314 s/iter. ETA=0:00:09


In [ ]:
import matplotlib.pyplot as plt

# Extract mAP scores from the evaluation metrics
map_50 = metrics["bbox"]["AP50"]  # mAP at IoU=0.50
map_75 = metrics["bbox"]["AP75"]  # mAP at IoU=0.75
map_all = metrics["bbox"]["AP"]   # mAP averaged over IoU thresholds (usually 0.50:0.95)

# Plot the mAP scores
plt.figure(figsize=(8, 6))
plt.bar(["mAP@0.50", "mAP@0.75", "mAP@0.50:0.95"], [map_50, map_75, map_all], color=['blue', 'orange', 'green'])
plt.title("Model Accuracy (mAP) at Different IoU Thresholds")
plt.ylabel("mAP")
plt.show()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/output/
